## Objective
This is a fully integrated prototype of a multi-agent system that supports the following:

1.   Conversational interface with limited memory
2.   Document-based Question Answering using RAG
3.   Text-to-image generation with prompt engineering
4.   Multi-agent task handling using a controller (Weather, SQL, Recommender)



In [6]:
!pip install langchain.openai
!pip install langchain_classic
!pip install langchain_community
!pip install langchain_core
!pip install langgraph_supervisor
# for RAG
!pip install faiss-cpu
!pip install pypdf
!pip install pymupdf


### Part 1. Conversational interface with limited memory
The assistant supports multi-turn, coherent dialogue with limited memory.

Part 1 demonstrates creating a RAG pipeline with LangChain framework. Following steps are included:
1. Loading a .pdf file
2. Use embedding model and LLM (GPT-4) from OpenAI
3. Retriveing response from FAISS vector database
4. Augmenting and generating the response

In [2]:
#1) RAG
# additional packages for RAG
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# Force-reinstall langchain to resolve persistent ModuleNotFoundError for langchain.memory
#!pip install --upgrade --force-reinstall langchain
import os
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
#from langchain.memory import ConversationBufferMemory
#from langchain_core.memory import BaseMemory
#from langchain_community.memory import ConversationBufferMemory

In [12]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
weather_api_key = userdata.get('WEATHER_API_KEY')

### Part 2. Document-based Question Answering using RAG

In [53]:
def rag_answer(query: str) -> str:
  print("In the function rag_answer")
  # load the pdf file
  file_path = "company_policy.pdf"
  #loader = PyMuPDFLoader(file_path)
  loader = PyPDFLoader(file_path)
  documents = loader.load()
  #documents

  # print number of documents loaded (24 pages in the pdf document)
  #print("Number of documents loaded: ", len(documents))
  # print(documents[0])
  # print(documents[0].page_content)

  # divide the document into chunks
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=500,
      chunk_overlap=100
  )
  document_chunks = text_splitter.split_documents(documents)
  #print("Number of chunks: ", len(document_chunks))

  # create embeddings for the chunks
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=openai_api_key)

  # print the model used for embeddings
  print("Embedding model: ", embeddings.model)

  # create FAISS vector store from the list of `Document` object and embeddings
  vector_store=FAISS.from_documents(document_chunks, embeddings)
  # save the faiss index to disk
  vector_store.save_local("fais_index")

  # create a ConversationalRetrievalChain chain
  # as_retriever() converts vector store into a retriever object
  retriever = vector_store.as_retriever(search_kwargs={"k":3})
  '''
  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      #memory=memory,
      retriever=retriever,
      return_source_documents=True,
  )
  '''
  # call the model
  llm = ChatOpenAI(model="gpt-4", api_key=openai_api_key, temperature=0)
  rag_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=retriever,
      return_source_documents=True,
  )

  #query = "What is the notice period?"
  result = rag_chain({"query":query})
  #result=rag_chain.run(query)
  #print("Question: ", query)
  #print("Answer: ", result['result'])
  #print("Source Documents:")
  #for doc in result['source_documents']:
  #  print("\nPage Content: ", doc.page_content[:500], ".....")
  return result


### Part 3. Image Generation with Prompt Engineering
Accepts text prompts and generates images using DALL·E/Replicate API. Includes prompt experimentation.

In [56]:
from openai import OpenAI
from IPython.display import Image

def generate_image(prompt: str) -> str:
  """
  Generate an image using DALL-E-3 and returns the URL

  """
  client = OpenAI(api_key=openai_api_key)
  response = client.images.generate(
      model="dall-e-3",
      prompt=prompt,
      #prompt="An astronaut in a rocketship in space with stars in the background",
      n=1,
      size="1024x1024"
  )
  image_url = response.data[0].url
  print("Image URL: ", image_url, "\n")
  Image(url=image_url, width=512)
  return image_url

def prompt_engineer_image(user_prompt: str) -> str:
  """
  Minimal prompt engineering to generate an image using DALL-E-3
  Appends a richer prompt to the user input for DALL-E-3
  """

  return (f"{user_prompt}. "
  "Highly detailed professional photography, clear lighting and sharp focus. "
  "Cinematic depth, good color balanceno watermark"
  )


### Part 4. Multi-Agent Coordination via Controller
Controller manages and routes tasks to agents (Weather, SQL, Recommender). Agents must collaborate to return coherent results.

*   Multi-agent reasoning system using LangGraph and LangGraph supervisor.
*   Agent routing, tool calling, and role-specific prompting are used.

In [18]:
# integrate all agents using a controller
# 3). Multi-agent system (LangGraph supervisor + ReAct agents)
# Agents include weather agent, SQL agent and recommender agent

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

agent_model=ChatOpenAI(model="gpt-4", temperature=0, api_key=openai_api_key)

In [19]:
# weather lookup
import requests
def weather_lookup(location):
  """ Fetches weather information for a given location"""
  try:
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {
        "key": weather_api_key,
        "q": location,
        "aqi": "no"
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()
  except requests.exceptions.RequestException as e:
    print("Exception occurred:", e)

weather_data = weather_lookup(location)
print(f"Location name: {weather_data['location']['name']}")
print(f"Temperature in degree Celsius : {weather_data['current']['temp_c']}")


Location name: Singapore
Temperature in degree Celsius : 31.1


In [85]:
import sqlite3
# Part 1: Setting Up the Database

def init_demo_db():
    conn = sqlite3.connect('company.db')
    c = conn.cursor()

    # Create sample tables
    c.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            salary REAL
        )
    ''')

    c.execute('''
        CREATE TABLE IF NOT EXISTS departments (
            id INTEGER PRIMARY KEY,
            name TEXT,
            budget REAL
        )
    ''')

    # Insert sample data
    c.execute("INSERT OR IGNORE INTO employees VALUES (1, 'John Doe', 'Engineering', 75000)")
    c.execute("INSERT OR IGNORE INTO employees VALUES (2, 'Jane Smith', 'Marketing', 65000)")
    c.execute("INSERT OR IGNORE INTO departments VALUES (1, 'Engineering', 1000000)")
    c.execute("INSERT OR IGNORE INTO departments VALUES (2, 'Marketing', 500000)")

    conn.commit()
    conn.close()

    # setup events database for recommender system

    """" The function setup_database() sets up the database connection using sqlite3 and
    creates database events.db containing the table events that stores events information.
    Events information includes the columns name, type, description, location, date.
    """

    conn = sqlite3.connect('events.db')
    c = conn.cursor()

    c.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            name TEXT,
            type TEXT,  -- 'indoor' or 'outdoor'
            description TEXT,
            location TEXT,
            date TEXT
        )
    ''')

    # Sample events
    events = [
        ('Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'),
        ('Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15'),
        ('Food Festival', 'outdoor', 'International cuisine', 'Waterfront', '2025-07-16'),
        ('Theater Show', 'indoor', 'Classical drama', 'Grand Theater', '2025-07-16')
    ]

    c.executemany('INSERT OR IGNORE INTO events (name, type, description, location, date) VALUES (?,?,?,?,?)', events)
    conn.commit()
    conn.close()
    print("Company database setup is completed.")
    conn = sqlite3.connect('company.db')
    c = conn.cursor()
    print("Employees table:")
    c.execute("SELECT * FROM employees")
    print(c.fetchall())
    print("\nDepartments table:")
    c.execute("SELECT * FROM departments")
    print(c.fetchall())
    conn.close()
    #print("events.db is created.")

#init_demo_db()


In [89]:
# SQL agent using OpenAI
import openai
import sqlite3
#from config import OPENAI_API_KEY
#openai.api_key = openai_api_key
client = openai.Client(api_key=openai_api_key)

# Part 2: Creating the SQL Generator
# define schema to guide SQL generation
def get_schema():
    return """
    Table: employees
    Columns:
    - id (INTEGER PRIMARY KEY)
    - name (TEXT)
    - department (TEXT)
    - salary (REAL)

    Table: departments
    Columns:
    - id (INTEGER PRIMARY KEY)
    - name (TEXT)
    - budget (REAL)
    """

# Generate SQL query from natural language
def generate_sql(question):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"""You are a SQL expert. Use this schema:\n{get_schema()}
            Return ONLY the SQL query without any explanation or markdown formatting."""
            },
            {"role": "user", "content": f"Generate SQL for: {question}"
            }
        ]
    )
    sql = response.choices[0].message.content.strip()

    # Remove any markdown code block syntax
    sql = sql.replace('```sql', '').replace('```SQL', '').replace('```', '')
    # Remove any explanatory text before or after the SQL
    sql_lines = [line.strip() for line in sql.split('\n') if line.strip()]
    sql = ' '.join(sql_lines)
    return sql

# Part 3: Implementing Query Execution
# validate SQL query before executing
def validate_sql(sql):
    # Basic safety checks

    sql_lower = sql.lower()
    if any(word in sql_lower for word in ['drop', 'delete', 'update', 'insert']):
        raise ValueError("Only SELECT queries are allowed")
    print("in validate_sql function", sql)
    return sql

# execute the SQL on SQLite database
def execute_query(sql):
    print("in execute_query function")
    sql = validate_sql(sql)
    conn = sqlite3.connect('company.db')
    print("successfully connected to company.db")
    try:
        cursor = conn.cursor()
        print("successfully created cursor")
        cursor.execute(sql)
        print("successfully executed query")
        results = cursor.fetchall()
        print("successfully fetched the results")
        print(results)
        return results
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        conn.close()
    print("sql results", results)

# format the SQL cleanly
def format_results(results):
    if not isinstance(results, list):
        return str(results)
    if not results:
        return "No results found"

        # If it's a single column result
    if len(results[0]) == 1:
        return "\n".join([str(row[0]) for row in results])

    # For multiple columns, try to format as a table
    # Get column names from the first result
    if isinstance(results[0], tuple):
        # Format each row with proper spacing
        formatted_rows = []
        for row in results:
            row_items = []
            for item in row:
                if isinstance(item, float):
                    row_items.append(f"${item:,.2f}" if "salary" in str(row) or "budget" in str(row) else f"{item:.2f}")
                else:
                    row_items.append(str(item))
            formatted_rows.append("\t".join(row_items))
        return "\n".join(formatted_rows)

    return "\n".join([str(row) for row in results])

# core agent: ask, generate, validate, run and respond
def run_sql(question):
  """Use this function to execute SQL query on SQLite and return rows as output text"""
  print("in run_sql function")
  try:
        # Generate SQL
        sql = generate_sql(question)
        print(f"Generated SQL: {sql}\n")
        validated_sql = validate_sql(sql)
        print(f"Validated SQL: {validated_sql}\n")
        # Execute and format results
        results = execute_query(validated_sql)

        print(f"Results: {results}\n")
        return format_results(results)
  except Exception as e:
        return f"Error: {str(e)}"

# Test the complete agent
'''
test_questions = [
    "What is the average salary in each department?",
    "Which department has the highest budget?",
    "List all employees earning more than 70000",
    "DROP TABLE employees"  # This should be caught by validation
]

for question in test_questions:
    print(f"\nQuestion: {question}")
    print(f"Answer: {run_sql(question)}")
    print("*" * 50)
'''

'\ntest_questions = [\n    "What is the average salary in each department?",\n    "Which department has the highest budget?",\n    "List all employees earning more than 70000",\n    "DROP TABLE employees"  # This should be caught by validation\n]\n\nfor question in test_questions:\n    print(f"\nQuestion: {question}")\n    print(f"Answer: {run_sql(question)}")\n    print("*" * 50)\n'

In [34]:
def recommend()

SyntaxError: expected ':' (ipython-input-4258005936.py, line 1)

In [73]:
# build agents
weather_agent = create_react_agent(
    model=agent_model,
    tools=[weather_lookup],
    name="weather_expert",
    prompt="You are a weather expert. Use the weather lookup. Give clear and concise answers."
)

sql_agent = create_react_agent(
    model=agent_model,
    tools=[run_sql],
    name="sql_expert",
    prompt=(
        "You are a SQL expert. Your sole purpose is to execute SQL queries using the 'run_sql' tool.\n"
        "ALWAYS use the 'run_sql' tool to answer questions related to the database using SQLite.\n"
        "Do NOT try to answer conversationally or provide SQL code directly. Only use the 'run_sql' tool.\n"
        "Provide precise answers based *only* on the tool's output.\n"
        f"The database schema is:\n{get_schema()}\n\n"
        "When you receive a user question about the database, your thought process MUST be:\n"
        "1. Identify the core question that requires an SQL query.\n"
        "2. Formulate the question for the 'run_sql' tool.\n"
        "3. Invoke the 'run_sql' tool with the formulated question."
    )
)

recommender_agent = create_react_agent(
    model=agent_model,
    tools=[],
    name="recommender_expert",
    prompt="You are a recommender expert. Ask questions only if absolutely necessary."
)

/tmp/ipython-input-2767118794.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  weather_agent = create_react_agent(
/tmp/ipython-input-2767118794.py:9: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  sql_agent = create_react_agent(
/tmp/ipython-input-2767118794.py:26: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  recommender_agent = create_react_agent(


In [93]:
# supervisor workflow
workflow = create_supervisor(
    [weather_agent, sql_agent, recommender_agent],
    model=agent_model,
    prompt=(
        "You are a supervisor for a team of specialized AI agents: weather_expert, sql_expert, and recommender_expert.\n"
        "Your primary role is to analyze the user's request and *unconditionally route* it to the most appropriate agent.\n"
        "- If the user asks for weather-related information, route the request to the 'weather_expert'.\n"
        "- If the user asks for SQL, database, table, or query-related information, *immediately route* the request to the 'sql_expert'.\n"
        "- If the user asks for event recommendation-related information, route the request to the 'recommender_expert'.\n"
        "After an agent has completed its task and returned a result, your ONLY job is to return that result directly to the user.\n"
        "Do NOT attempt to answer the user yourself. Your ONLY job is to select the correct agent for the task and return its output."
    )
)

supervisor_app = workflow.compile()

def multi_agent_route(user_request: str) -> str:
  """ This is the single entry point of multi-agent sub-system
      Run via supervisor and return final agent response.
  """
  print("in multi_agent_route")
  result = supervisor_app.invoke({
      "messages": [{"role": "user", "content": user_request}]
  })

  messages = result.get("messages", [])
  # Iterate backwards to find the last message with actual content
  for msg in reversed(messages):
      # Langchain messages have a 'content' attribute. Check if it exists and is not empty.
      if hasattr(msg, 'content') and msg.content:
          return msg.content
      # Fallback for dictionary-like messages, check 'content' key
      if isinstance(msg, dict) and msg.get('content'):
          return msg.get('content')

  # Fallback if no content found in any message
  if messages:
      # If there are messages but none explicitly matched, return the string representation of the last message
      return str(messages[-1])
  else:
      print("No response from multi-agent system")
      return "No response from multi-agent system" # Ensure a string is always returned

In [38]:
#4) Controller
# import necessary packages
from langchain_openai import ChatOpenAI as ControllerChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
#from langchain.memory import ConversationBufferMemory
from langchain_classic.memory import ConversationBufferMemory

In [47]:
# wrap features as tools so controller can use them
@tool
def rag_tool(question: str) -> str:
  """ Use this if the the user asks questions related to the pdf document (RAG). """
  #rag_output = rag_chain({"query":question})
  rag_output = rag_answer(question)

  # return answer along with source for proof
  lines = [rag_output['result']]
  if rag_output['source_documents']:
    lines.append("\nSources:")
    # meta often contains metadata about the retrieved document chunk, and
    #'source' would typically be the filename or URL where the information originated.
    for doc in rag_output['source_documents']:
      source_preview = doc.page_content.strip().replace("\n", " ")[:180]
      # The score is not directly available when iterating over source_documents as returned by RetrievalQA.
      lines.append(f"{doc.metadata.get('source', 'N/A')}: {source_preview}...")
  return "\n".join(lines)

@tool
def image_tool(user_prompt: str) -> str:
  """ Use this tool if the user wants to generate image from text."""
  engineered_prompt = prompt_engineer_image(user_prompt)
  image_url = generate_image(engineered_prompt)
  return (
      "Prompt engineering is applied. \n"
      f"Engineered prompt: {engineered_prompt}\n"
      f"Image URL: {image_url}"
  )

@tool
def multi_agent_tool(user_request: str) -> str:
  """ Use this tool if the user asks about weather or SQL/database query or event recommendation"""
  return multi_agent_route(user_request)

In [ ]:
# start here
# controller llm
controller_llm = ControllerChatOpenAI(model="gpt-4", temperature=0, api_key=openai_api_key)

# limited memory snippet
# define memory to maintain context throughout human interaction
memory = ConversationBufferMemory(
    # key under which conversation history will be stored
    memory_key="chat_history",
    # return full messge objects (including role and content)
    return_messages=True,
    )

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools if necessary to answer user query. If you use a tool and it provides a direct answer, ONLY return the tool's output. Do NOT add any conversational text or summary when a tool's direct output is available."),
    "Tool use rules:\n",
    "If the user wants to generate an image, use image_tool\n",
    "If the user asks a question about pdf document, use rag_tool\n",
    "If the users asks about weather/sql/recommendation, use multi_agent_tool\n"
    "Otherwise answer as usual\n",
    # placeholder that will be filled with the conversation history. The variable_name="chat_history"
    # means that the memory component (defined earlier as memory) will inject the past conversation
    # turns into the prompt at this point.
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{user_input}"),
    # The agent_scratchpad is where the agent internally keeps track of its thoughts,
    # the tools it decided to use, the observations it got back from tools, and its
    # planning steps before generating a final response.
    ("placeholder", "{agent_scratchpad}")
]
)

tools = [rag_tool, image_tool, multi_agent_tool]

agent = create_tool_calling_agent(
    llm=controller_llm,
    tools=tools,
    prompt=prompt
)

agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

#5). Demo run
def print_menu():
  print("*"*50)
  print("Integrated AI assistant menu")
  print("*"*50)
  print("1. General conversation (chat with memeory)")
  print("2. Ask question from .pdf document (RAG)")
  print("3. Generate imge from text")
  print("4. Weather information")
  print("5. Run SQL query")
  print("6. Get recommendations")
  print("0. Exit")
  print("*"*50)

def main():
  init_demo_db()
  print("\nIntegratd Prototype Started (Menu Drive)")
  print("All user requests are routed only through the Controller\n")

  while True:
    print_menu()
    choice = input("Enter your choice (0-6): ").strip()

    if choice == "0":
      print("\nExiting system. Goodbye")
      break
    elif choice == "1":
      user_input = input("\nEnter your message: ")
      controller_input = user_input
    elif choice == "2":
      question = input("\nEnter your document based question: ")
      controller_input = f"Answer this question from .pdf document: {question}"
    elif choice == "3":
      image_prompt = input("\nEnter your description for the image: ")
      controller_input = f"Generate image from this description: {image_prompt}"
    elif choice == "4":
      location = input("\nEnter the location for weather: ")
      controller_input = f"What is the temperature in {location} today?"
    elif choice == "5":
      sql_query = input("\nEnter SQL query: ")
      controller_input = f"Run the SQL query and show the results:\n{sql_query}"
    elif choice == "6":
      item = input("\nwhat recommendation do you want?")
      constraints= input("Any constraints?")
      controller_input = (
          f"Recommend{item}. Constraints {constraints}"
          if constraints else f"Recoommend{item}"
      )
    else:
      print("\nInvalid choice. Please try again.")
      continue

    response = agent_executor.invoke({"user_input": controller_input})
    print(response["output"])
    print("\n" + "-"*50)
    print("Assistant Response:")
    print("\n" + "-"*50)
    print(response["output"])
    print("\n" + "-"*50)

main()

Company database setup is completed.
Employees table:
[(1, 'John Doe', 'Engineering', 75000.0), (2, 'Jane Smith', 'Marketing', 65000.0)]

Departments table:
[(1, 'Engineering', 1000000.0), (2, 'Marketing', 500000.0)]

Integratd Prototype Started (Menu Drive)
All user requests are routed only through the Controller

**************************************************
Integrated AI assistant menu
**************************************************
1. General conversation (chat with memeory)
2. Ask question from .pdf document (RAG)
3. Generate imge from text
4. Weather information
5. Run SQL query
6. Get recommendations
0. Exit
**************************************************


> Entering new AgentExecutor chain...

Invoking: `multi_agent_tool` with `{'user_request': 'What is the average salary in each department'}`


in multi_agent_route
in run_sql function
Generated SQL: SELECT department, AVG(salary) as average_salary FROM employees GROUP BY department

in validate_sql function SELECT 